In [1]:
%matplotlib qt
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from copy import copy

from gauss import Gaussian
from Estimator_EKF import EKF

dt = 1
xt = np.array(np.zeros((3,11)))
zt = np.array(np.zeros((3,11)))

xt[:,0] = np.array([0,0,0])

u = np.array(np.zeros((2,10)))

wt = (2*np.pi)/20

R = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.05]])
Q = np.array([[0.05,0,0],[0,0.05,0],[0,0,0.05]])

mov_noise_mean = np.array([0,0,0])
mes_noise_mean = np.array([0,0,0])

mov_noise = np.random.multivariate_normal(mov_noise_mean, R)        
xt[:,0] = xt[:,0] + mov_noise
    
mes_noise = np.random.multivariate_normal(mes_noise_mean, Q)
zt[:,0] = xt[:,0] + mes_noise

for i in range(1, len(xt[0])):
    u[0, i-1] = np.random.normal(2, 0.5)
    u[1, i-1] = np.random.normal(wt,0)
    
    x_prev = xt[0, i-1]
    y_prev = xt[1, i-1]
    theta_prev = xt[2, i-1]
    
    v = u[0,i-1]
    w = u[1,i-1]
   
    x_t = x_prev - (v/w)*np.sin(theta_prev) + (v/w)*np.sin(theta_prev + w*dt)
    y_t = y_prev + (v/w)*np.cos(theta_prev) - (v/w)*np.cos(theta_prev + w*dt)
    theta_t = theta_prev + w*dt
    
    mov_noise = np.random.multivariate_normal(mov_noise_mean, R)        
    xt[:,i] = np.array([x_t,y_t,theta_t])+mov_noise
    
    mes_noise = np.random.multivariate_normal(mes_noise_mean, Q)
    zt[:,i] = xt[:,i] + mes_noise
    #print(xt[:,i-1])


   

In [2]:
##EKF##
myEKF = EKF(R, Q)

estimatedMeans = np.array(np.zeros((3,11)))
estimatedSigs = np.array(np.zeros((3,3,11)))

estimatedSigs[:,:,0] = np.array([[0.1, 0, 0],[0, 0.1, 0], [0, 0, 0.1]])

estimatedLambdas = np.array(np.zeros((3,11)))
estimatedVs = np.array(np.zeros((3,3,11)))

fig0 = plt.figure(0)
ax0 = plt.subplot(111, aspect = 'equal')

for i in range (0, len(u[0])):
    estimatedMeans[:, i+1], estimatedSigs[:,:,i+1] = myEKF.func_EKF(estimatedMeans[:, i], estimatedSigs[:,:,i] , u[:, i], zt[:, i+1])
    
for i in range (0, 11):
    estimatedLambdas[:, i], estimatedVs[:,:, i] = np.linalg.eig(estimatedSigs[:,:,i])
    estimatedLambdas[:, i] = np.sqrt(estimatedLambdas[:, i])
    
    for j in range(3, 4):
        ell = Ellipse(xy = (estimatedMeans[0, i],estimatedMeans[1, i]), 
                        width = estimatedLambdas[0, i]*j*2, 
                        height = estimatedLambdas[1, i]*j*2, 
                        angle = np.rad2deg(np.arccos(estimatedVs[0,0,i])), color = 'black')
            
        ell.set_facecolor('none')
        ax0.add_artist(ell)

plt.scatter(xt[0,:], xt[1,:], c = "red", marker = "x")
plt.scatter(zt[0,:], zt[1,:], c = "blue", marker = ".")

i = 0
for x, y in zip(xt[0,:], xt[1,:]):
    plt.text(x, y, str(i), color="black", fontsize=12)
    i =i+1
plt.margins(0.1)
#    plt.scatter(zt[0], zt[1], c = "green", marker = '.')
plt.show()

In [3]:
##UKF##

